# TRABAJO PARCIAL: QUORIDOR



## 1. INTRODUCCION

Como trabajo parcial, para el curso de complejidad algorítmica, se implementará el juego Quoridor en el lenguaje de programación Python. El juego consiste en llegar al extremo opuesto de tablero antes que los demás jugadores, para ello los jugadores podrán poner barreras con el objetivo de bloquear los caminos de los demás jugadores y así poder llegar antes al extremo opuesto. Además, los jugadores solo pueden moverse un solo espacio (arriba, abajo, derecha o izquierda), saltar encima de una ficha, si la ficha del otro jugador esta al costado), moverse en diagonal si al querer saltar encima de una ficha existe una barrera en el siguiente espacio o poner una barrera. Para el desarrollo de este proyecto se usará Jupyter Notebook el cual se usará para explicar detalladamente el código a realizar. También se usará GitHub para poder tener un correcto control de versiones y poder trabajar de manera colaborativa. Por último, se usará la plataforma Discord para comunicarnos como equipo.

## 2. ESTADO DEL ARTE

Para el apoyo de la implementación de nuestra solución, hemos recolectado una serie de investigaciones que evidencian las posibles soluciones del juego. Por cada investigación, nos fijaremos únicamente en cómo se definió la tabla del juego y la inserción de barreras (obstáculos del juego) y qué algoritmos fueron utilizados para la implementación del pathfinding y la inserción de barreras.

### Primera solución: Monte Carlo Tree Search for Quoridor


Fuente: Massagué Respall, Victor & Brown, Joseph & Aslam, Hamna. (2018). Monte Carlo Tree Search for Quoridor. 

#### Tabla del juego 
"The class Board has all the information related to theboard game, squares, fences and both players. Allsquares are stored in an ArrayList, and to navigatethrough the board, we use a Map that each key is asquare and it returns a Set of all adjacent squares tothe key"

#### Barreras
"For checking if a certain fence can be placed, we initially start the game with a Set containing all possible placesthat a fence can be placed on the board. Then, each time a fence is placed, we update the Set removing ”forbid-den” positions. After checking that the position of that fence is inside the Set, we must check that both playersare not completely blocked from reaching the opposing baseline. The system explained before for storing fences is alsouseful for returning all possible movements available tothe player."

#### Algoritmos
"The Monte Carlo Tree Search algorithm is used forbuilding the AI agent for Quoridor, as it appears to bean eﬃcient algorithm for this type of board game andgame tree size. It is a probabilistic search algorithmwith a unique decision-making ability because of its ef-ﬁciency in open-ended environments with an enormousamount of possibilities. To deal with the size of thegame tree, it applies Monte Carlo method (Metropolisand Ulam 1949). As it is based on random sampling ofgame states, it does not need to use brute force.We have built a game tree with a root node, then it isexpanded with random simulations. In the process, wemaintain the number of times we have visited a speciﬁcnode and a win score, used to evaluate the state of theboard. In the end, we select the node with best resultsand higher win scores."

## 3. Metodología

Para el desarrollo del juego se usaran grafos el cual nos permitira saber cuales son los espacios aledaños a una ficha. Por otro lado se hara uso del algoritmo dijistra para poder encontrar el camino minimo para que la ficha llegue al extremo opuesto de donde empezo. Para ello inicialmente se usaran tableros de 3x3 con el objetivo de probar los algoritmos realizados. Luego se usaran tableros de 5x5 y 7x7 para poser vizaulizar un grafico de la complejidad.

## 4. EXPERIMENTOS 

## 5. RESULTADOS

## 6. CONCLUSIONES

## 7. PRESENTACION